# Stress analysis for each element 

In [22]:
name = 'yannis'

# Import modules 

In [23]:
import pandas as pd
import sys 
import os
sys.path.insert(0, os.path.abspath('..'))
sys.path.insert(0, os.path.abspath('../formulas'))

from formulas.panels import *
from formulas.helpers import *
from formulas.abd_matrix import *
from formulas.Strength import * 

# Import needed files 

## Import panel stresses

In [24]:
PlyStressesX = pd.read_csv(f'../data/{name}/compositePanelXX.csv')
PlyStressesX = PlyStressesX.drop(columns=['FileID', 'Step'])
PlyStressesY = pd.read_csv(f'../data/{name}/compositePanelYY.csv')
PlyStressesY = PlyStressesY.drop(columns=['FileID', 'Step'])
PlyStressesXY = pd.read_csv(f'../data/{name}/compositePanelXY.csv')
PlyStressesXY = PlyStressesXY.drop(columns=['FileID', 'Step'])
PlyStressesX

,Elements,Loadcase,Layer,Composite Stresses:Normal X Stress
0,1,1,Ply 1,-30.129749
1,2,1,Ply 1,33.933514
2,3,1,Ply 1,5.449137
3,4,1,Ply 1,-13.714887
4,5,1,Ply 1,25.343819
...,...,...,...,...
1435,26,3,Ply 9,19.149086
1436,27,3,Ply 9,-22.105305
1437,28,3,Ply 9,64.143097
1438,29,3,Ply 9,17.373724


In [25]:
PlyStressesPanel=pd.merge(PlyStressesX,PlyStressesY, on=['Elements', 'Layer', 'Loadcase'])
PlyStressesPanel = pd.merge(PlyStressesPanel, PlyStressesXY, on=['Elements', 'Layer', 'Loadcase'])
PlyStressesPanel = PlyStressesPanel.rename(columns={'Composite Stresses:Normal X Stress':'Normal_1', 'Composite Stresses:Normal Y Stress':'Normal_2', 'Composite Stresses:Shear XY Stress':'Shear_12' })
PlyStressesPanel

,Elements,Loadcase,Layer,Normal_1,Normal_2,Shear_12
0,1,1,Ply 1,-30.129749,-10.697798,11.398685
1,2,1,Ply 1,33.933514,-4.099037,4.942441
2,3,1,Ply 1,5.449137,-11.857535,12.995567
3,4,1,Ply 1,-13.714887,-10.444026,8.897148
4,5,1,Ply 1,25.343819,-5.437790,4.564201
...,...,...,...,...,...,...
1435,26,3,Ply 9,19.149086,-2.126457,-14.955009
1436,27,3,Ply 9,-22.105305,-2.504914,-15.524483
1437,28,3,Ply 9,64.143097,-1.380899,-15.699356
1438,29,3,Ply 9,17.373724,-2.618130,-12.505588


## Import stringer plies 

In [26]:
StringerStrains = pd.read_csv(f'../data/{name}/stringer_strain.csv')
StringerStrains = StringerStrains.drop(columns=['FileID', 'Step'])
StringerStrains = StringerStrains.rename(columns={'Element Strains (1D):CBAR/CBEAM Axial Strain':'strainX'})
StringerStrains

,Elements,Loadcase,strainX
0,40,1,-0.001241
1,41,1,-0.001524
2,42,1,-0.001251
3,46,1,-0.001285
4,47,1,-0.001590
5,48,1,-0.001280
6,52,1,-0.001293
7,53,1,-0.001595
8,54,1,-0.001277
9,58,1,-0.001251


# Constants for analysis 

In [27]:
# r = perpendicular 
# p = parallel
# c = compression 
# t = tension 
# Resistance values 

R_p_t = 3050
R_p_c = 1500
R_r_c = 50
R_r_t = 300 
R_rp = 100

# p values 
# Perpendicular, parallel
p_rp_c = 0.25
p_rp_t = 0.25

# Perpendicular, perpendicular
p_rr_c = 0.25
p_rr_t = 0.25

#Ply orientation in flange 
StringerFlange=[45,45,-45,-45,0,0,90,90,90,90,0,0,-45,-45,45,45]

# Personal data Provider 

In [28]:
personal_data = personal_data_provider(name)
E_11 = personal_data[0]
E_22 = personal_data[1]
G_12 = personal_data[2]
nu_12 = personal_data[3]

# Calculate RF and mode for panels

In [29]:
PlyStressesPanel[['Mode', 'RF_IFF', 'RF_FF']] = PlyStressesPanel.apply(strength, R_p_t=R_p_t, R_p_c=R_p_c, R_r_c=R_r_c, R_r_t=R_r_t, 
                                                    R_rp=R_rp, p_rp_c=p_rp_c, p_rp_t=p_rp_t, p_rr_c=p_rr_c, p_rr_t=p_rr_t,
                                                    axis=1, result_type='expand')
PlyStressesPanel

,Elements,Loadcase,Layer,Normal_1,Normal_2,Shear_12,Mode,RF_IFF,RF_FF
0,1,1,Ply 1,-30.129749,-10.697798,11.398685,C,4.470825,49.784682
1,2,1,Ply 1,33.933514,-4.099037,4.942441,C,11.527610,89.881644
2,3,1,Ply 1,5.449137,-11.857535,12.995567,C,4.023417,559.721637
3,4,1,Ply 1,-13.714887,-10.444026,8.897148,C,4.652374,109.370207
4,5,1,Ply 1,25.343819,-5.437790,4.564201,C,8.942898,120.344927
...,...,...,...,...,...,...,...,...,...
1435,26,3,Ply 9,19.149086,-2.126457,-14.955009,B,6.928643,159.276532
1436,27,3,Ply 9,-22.105305,-2.504914,-15.524483,B,6.706512,67.857015
1437,28,3,Ply 9,64.143097,-1.380899,-15.699356,B,6.511296,47.549934
1438,29,3,Ply 9,17.373724,-2.618130,-12.505588,C,9.985093,175.552461


## Calculate RF_strength 

In [30]:
PlyStressesPanel['RF_strength'] = PlyStressesPanel[['RF_IFF', 'RF_FF']].min(axis=1)
PlyStressesPanel.head(5)

,Elements,Loadcase,Layer,Normal_1,Normal_2,Shear_12,Mode,RF_IFF,RF_FF,RF_strength
0,1,1,Ply 1,-30.129749,-10.697798,11.398685,C,4.470825,49.784682,4.470825
1,2,1,Ply 1,33.933514,-4.099037,4.942441,C,11.527610,89.881644,11.527610
2,3,1,Ply 1,5.449137,-11.857535,12.995567,C,4.023417,559.721637,4.023417
3,4,1,Ply 1,-13.714887,-10.444026,8.897148,C,4.652374,109.370207,4.652374
4,5,1,Ply 1,25.343819,-5.437790,4.564201,C,8.942898,120.344927,8.942898


## Convert to output for panels 

In [31]:
# We can drop the stresses 
PlyStressesPanel = PlyStressesPanel.drop(columns=['Normal_1', 'Normal_2', 'Shear_12'])

#Reorder plies 
PlyStressesPanel['Layer_num'] = PlyStressesPanel['Layer'].str.extract('(\d+)').astype(int)
PlyStressesPanel = PlyStressesPanel.sort_values(by='Layer_num')
PlyStressesPanel = PlyStressesPanel.drop(columns=['Layer'])
PlyStressesPanel = PlyStressesPanel.rename(columns={'Layer_num':'Layer'})

# Resort columns to desired order
PlyStressesPanel = PlyStressesPanel[['Elements', 'Layer', 'Loadcase', 'RF_FF', 'RF_IFF', 'Mode', 'RF_strength']]

# We need to analyse element 8 
PlyStressesPanel = PlyStressesPanel[PlyStressesPanel['Elements'] == 8]
#Now we separate the loadcases 
PlyStressesPanelLC1 = PlyStressesPanel[PlyStressesPanel['Loadcase'] == 1]
PlyStressesPanelLC2 = PlyStressesPanel[PlyStressesPanel['Loadcase'] == 2]
PlyStressesPanelLC3 = PlyStressesPanel[PlyStressesPanel['Loadcase'] == 3]
PlyStressesPanelLC1

<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Yanni\AppData\Local\Temp\ipykernel_18156\2736637046.py:5: SyntaxWarning: invalid escape sequence '\d'
  PlyStressesPanel['Layer_num'] = PlyStressesPanel['Layer'].str.extract('(\d+)').astype(int)


,Elements,Layer,Loadcase,RF_FF,RF_IFF,Mode,RF_strength
7,8,1,1,102.592367,9.241452,C,9.241452
247,8,2,1,103.547772,9.125119,C,9.125119
277,8,3,1,19.041854,18.155291,A,18.155291
307,8,4,1,18.864378,17.842834,A,17.842834
337,8,5,1,15.355873,21.888509,A,15.355873
367,8,6,1,15.068526,21.776240,A,15.068526
397,8,7,1,63.870059,7.427025,C,7.427025
427,8,8,1,62.756509,7.295658,C,7.295658
457,8,9,1,61.681122,7.168814,C,7.168814
37,8,10,1,60.641969,7.046265,C,7.046265


## Rename Loadcases 

In [32]:
# Loadcase 1
PlyStressesPanelLC1 = PlyStressesPanelLC1.rename(columns={'RF_FF':'R_FF_LC1', 'RF_IFF':'R_IFF_LC1', 'Mode':'Mode_LC1', 'RF_strength':'RF_strength_LC1'})
PlyStressesPanelLC1 = PlyStressesPanelLC1.reset_index()
PlyStressesPanelLC1 = PlyStressesPanelLC1.drop(columns=['index', 'Loadcase'])

# Loadcase 2
PlyStressesPanelLC2 = PlyStressesPanelLC2.rename(columns={'RF_FF':'R_FF_LC2', 'RF_IFF':'R_IFF_LC2', 'Mode':'Mode_LC2', 'RF_strength':'RF_strength_LC2'})
PlyStressesPanelLC2 = PlyStressesPanelLC2.reset_index()
PlyStressesPanelLC2 = PlyStressesPanelLC2.drop(columns=['index', 'Loadcase'])

#Loadcase 3 
PlyStressesPanelLC3 = PlyStressesPanelLC3.rename(columns={'RF_FF':'R_FF_LC3', 'RF_IFF':'R_IFF_LC3', 'Mode':'Mode_LC3', 'RF_strength':'RF_strength_LC3'})
PlyStressesPanelLC3 = PlyStressesPanelLC3.reset_index()
PlyStressesPanelLC3 = PlyStressesPanelLC3.drop(columns=['index', 'Loadcase'])
PlyStressesPanelLC1

,Elements,Layer,R_FF_LC1,R_IFF_LC1,Mode_LC1,RF_strength_LC1
0,8,1,102.592367,9.241452,C,9.241452
1,8,2,103.547772,9.125119,C,9.125119
2,8,3,19.041854,18.155291,A,18.155291
3,8,4,18.864378,17.842834,A,17.842834
4,8,5,15.355873,21.888509,A,15.355873
5,8,6,15.068526,21.776240,A,15.068526
6,8,7,63.870059,7.427025,C,7.427025
7,8,8,62.756509,7.295658,C,7.295658
8,8,9,61.681122,7.168814,C,7.168814
9,8,10,60.641969,7.046265,C,7.046265


## Combine Loadcases 

In [33]:
PlyStressesPanel = PlyStressesPanelLC1.merge(PlyStressesPanelLC2, on=['Elements', 'Layer'], suffixes=('_LC1', '_LC2'))
PlyStressesPanel = PlyStressesPanel.merge(PlyStressesPanelLC3, on=['Elements', 'Layer'])
PlyStressesPanel

,Elements,Layer,R_FF_LC1,R_IFF_LC1,Mode_LC1,RF_strength_LC1,R_FF_LC2,R_IFF_LC2,Mode_LC2,RF_strength_LC2,R_FF_LC3,R_IFF_LC3,Mode_LC3,RF_strength_LC3
0,8,1,102.592367,9.241452,C,9.241452,166.069983,7.614906,C,7.614906,14.242667,4.267838,C,4.267838
1,8,2,103.547772,9.125119,C,9.125119,171.422072,7.503888,C,7.503888,14.248498,4.228740,C,4.228740
2,8,3,19.041854,18.155291,A,18.155291,16.162334,33.699813,C,16.162334,6.758993,21.101817,A,6.758993
3,8,4,18.864378,17.842834,A,17.842834,15.973838,32.981576,C,15.973838,6.716952,21.038541,A,6.716952
4,8,5,15.355873,21.888509,A,15.355873,17.245267,25.573887,C,17.245267,53.166992,5.772153,A,5.772153
5,8,6,15.068526,21.776240,A,15.068526,17.008993,24.990682,C,17.008993,51.156081,5.759720,A,5.759720
6,8,7,63.870059,7.427025,C,7.427025,427.580219,7.383455,C,7.383455,184.091562,5.957797,B,5.957797
7,8,8,62.756509,7.295658,C,7.295658,457.075890,7.273559,C,7.273559,188.269224,5.947783,B,5.947783
8,8,9,61.681122,7.168814,C,7.168814,490.942464,7.166860,C,7.166860,192.640935,5.937794,B,5.937794
9,8,10,60.641969,7.046265,C,7.046265,530.229262,7.063221,C,7.063221,197.220475,5.927829,B,5.927829


# Output panel 

In [34]:
PlyStressesPanel.to_excel(f'../data/{name}/output/panelStrength.xlsx')

# Calculate stringer strength 

In [35]:
StringerStrains.head(5)

,Elements,Loadcase,strainX
0,40,1,-0.001241
1,41,1,-0.001524
2,42,1,-0.001251
3,46,1,-0.001285
4,47,1,-0.001590


## Expand the dataframe 

In [36]:
# Create a DataFrame with PlyNumber 1-16
ply_numbers = pd.DataFrame({'PlyNumber': np.arange(1, 17),
                            'plyTheta': StringerFlange})

# Cross join to expand each element row to 16 plies
StringerStrains = StringerStrains.merge(ply_numbers, how='cross')
#StringerStrains = StringerStrains.drop(columns=['PlyNumber_x'])
StringerStrains.head(33)

,Elements,Loadcase,strainX,PlyNumber,plyTheta
0,40,1,-0.001241,1,45
1,40,1,-0.001241,2,45
2,40,1,-0.001241,3,-45
3,40,1,-0.001241,4,-45
4,40,1,-0.001241,5,0
5,40,1,-0.001241,6,0
6,40,1,-0.001241,7,90
7,40,1,-0.001241,8,90
8,40,1,-0.001241,9,90
9,40,1,-0.001241,10,90


In [37]:
StringerStrains[['Normal_1', 'Normal_2', 'Shear_12']] = StringerStrains.apply(
    calculateMatStress, EModulus1=E_11, EModulus2=E_22, ShearModulus=G_12, axis=1,
    result_type='expand'
)
StringerStrains

,Elements,Loadcase,strainX,PlyNumber,plyTheta,Normal_1,Normal_2,Shear_12
0,40,1,-0.001241,1,45,-76.533936,-7.636167,5.693383
1,40,1,-0.001241,2,45,-76.533936,-7.636167,5.693383
2,40,1,-0.001241,3,-45,-76.533936,-7.636167,-5.693383
3,40,1,-0.001241,4,-45,-76.533936,-7.636167,-5.693383
4,40,1,-0.001241,5,0,-149.278495,-3.789376,0.000000
...,...,...,...,...,...,...,...,...
571,60,3,-0.000378,12,0,-45.542594,-1.156081,0.000000
572,60,3,-0.000378,13,-45,-23.349337,-2.329678,-1.736964
573,60,3,-0.000378,14,-45,-23.349337,-2.329678,-1.736964
574,60,3,-0.000378,15,45,-23.349337,-2.329678,1.736964


## Now calculate the FF and the IFF 

In [38]:
StringerStrains[['Mode', 'RF_IFF', 'RF_FF']] = StringerStrains.apply(strength, R_p_t=R_p_t, R_p_c=R_p_c, R_r_c=R_r_c, R_r_t=R_r_t, 
                                                    R_rp=R_rp, p_rp_c=p_rp_c, p_rp_t=p_rp_t, p_rr_c=p_rr_c, p_rr_t=p_rr_t,
                                                    axis=1, result_type='expand')
StringerStrains

c:\Users\Yanni\No_OneDrive\ASE Project\Task2\tum-ase-project-2025-2\formulas\Strength.py:78: RuntimeWarning: divide by zero encountered in scalar divide
  elif abs(row['Normal_2']/row['Shear_12']) <= criterion:


,Elements,Loadcase,strainX,PlyNumber,plyTheta,Normal_1,Normal_2,Shear_12,Mode,RF_IFF,RF_FF
0,40,1,-0.001241,1,45,-76.533936,-7.636167,5.693383,C,6.405360,19.599149
1,40,1,-0.001241,2,45,-76.533936,-7.636167,5.693383,C,6.405360,19.599149
2,40,1,-0.001241,3,-45,-76.533936,-7.636167,-5.693383,C,6.405360,19.599149
3,40,1,-0.001241,4,-45,-76.533936,-7.636167,-5.693383,C,6.405360,19.599149
4,40,1,-0.001241,5,0,-149.278495,-3.789376,0.000000,C,13.194784,10.048333
...,...,...,...,...,...,...,...,...,...,...,...
571,60,3,-0.000378,12,0,-45.542594,-1.156081,0.000000,C,43.249569,32.936200
572,60,3,-0.000378,13,-45,-23.349337,-2.329678,-1.736964,C,20.995347,64.241652
573,60,3,-0.000378,14,-45,-23.349337,-2.329678,-1.736964,C,20.995347,64.241652
574,60,3,-0.000378,15,45,-23.349337,-2.329678,1.736964,C,20.995347,64.241652


In [39]:
StringerStrains['RF_strength'] = StringerStrains[['RF_IFF', 'RF_FF']].min(axis=1)
StringerStrains.head(5)

,Elements,Loadcase,strainX,PlyNumber,plyTheta,Normal_1,Normal_2,Shear_12,Mode,RF_IFF,RF_FF,RF_strength
0,40,1,-0.001241,1,45,-76.533936,-7.636167,5.693383,C,6.405360,19.599149,6.405360
1,40,1,-0.001241,2,45,-76.533936,-7.636167,5.693383,C,6.405360,19.599149,6.405360
2,40,1,-0.001241,3,-45,-76.533936,-7.636167,-5.693383,C,6.405360,19.599149,6.405360
3,40,1,-0.001241,4,-45,-76.533936,-7.636167,-5.693383,C,6.405360,19.599149,6.405360
4,40,1,-0.001241,5,0,-149.278495,-3.789376,0.000000,C,13.194784,10.048333,10.048333


In [40]:
# We can drop the stresses 
StringerStrains = StringerStrains.drop(columns=['Normal_1', 'Normal_2', 'Shear_12','strainX', 'plyTheta'])

# Resort columns to desired order
StringerStrains = StringerStrains[['Elements', 'PlyNumber', 'Loadcase', 'RF_FF', 'RF_IFF', 'Mode', 'RF_strength']]

# We need to analyse element 8 
StringerStrains = StringerStrains[StringerStrains['Elements'] == 60]
#Now we separate the loadcases 
PlyStressesStringerLC1 = StringerStrains[StringerStrains['Loadcase'] == 1]
PlyStressesStringerLC2 = StringerStrains[StringerStrains['Loadcase'] == 2]
PlyStressesStringerLC3 = StringerStrains[StringerStrains['Loadcase'] == 3]
PlyStressesStringerLC1

,Elements,PlyNumber,Loadcase,RF_FF,RF_IFF,Mode,RF_strength
176,60,1,1,20.110431,6.572457,C,6.572457
177,60,2,1,20.110431,6.572457,C,6.572457
178,60,3,1,20.110431,6.572457,C,6.572457
179,60,4,1,20.110431,6.572457,C,6.572457
180,60,5,1,10.310463,13.538996,C,10.310463
181,60,6,1,10.310463,13.538996,C,10.310463
182,60,7,1,406.169882,4.467869,C,4.467869
183,60,8,1,406.169882,4.467869,C,4.467869
184,60,9,1,406.169882,4.467869,C,4.467869
185,60,10,1,406.169882,4.467869,C,4.467869


## Rename Loadcases 

In [41]:
# Loadcase 1
PlyStressesStringerLC1 = PlyStressesStringerLC1.rename(columns={'RF_FF':'R_FF_LC1', 'RF_IFF':'R_IFF_LC1', 'Mode':'Mode_LC1', 'RF_strength':'RF_strength_LC1'})
PlyStressesStringerLC1 = PlyStressesStringerLC1.reset_index()
PlyStressesStringerLC1 = PlyStressesStringerLC1.drop(columns=['index', 'Loadcase'])

# Loadcase 2
PlyStressesStringerLC2 = PlyStressesStringerLC2.rename(columns={'RF_FF':'R_FF_LC2', 'RF_IFF':'R_IFF_LC2', 'Mode':'Mode_LC2', 'RF_strength':'RF_strength_LC2'})
PlyStressesStringerLC2 = PlyStressesStringerLC2.reset_index()
PlyStressesStringerLC2 = PlyStressesStringerLC2.drop(columns=['index', 'Loadcase'])

#Loadcase 3 
PlyStressesStringerLC3 = PlyStressesStringerLC3.rename(columns={'RF_FF':'R_FF_LC3', 'RF_IFF':'R_IFF_LC3', 'Mode':'Mode_LC3', 'RF_strength':'RF_strength_LC3'})
PlyStressesStringerLC3 = PlyStressesStringerLC3.reset_index()
PlyStressesStringerLC3 = PlyStressesStringerLC3.drop(columns=['index', 'Loadcase'])
PlyStressesStringerLC1

,Elements,PlyNumber,R_FF_LC1,R_IFF_LC1,Mode_LC1,RF_strength_LC1
0,60,1,20.110431,6.572457,C,6.572457
1,60,2,20.110431,6.572457,C,6.572457
2,60,3,20.110431,6.572457,C,6.572457
3,60,4,20.110431,6.572457,C,6.572457
4,60,5,10.310463,13.538996,C,10.310463
5,60,6,10.310463,13.538996,C,10.310463
6,60,7,406.169882,4.467869,C,4.467869
7,60,8,406.169882,4.467869,C,4.467869
8,60,9,406.169882,4.467869,C,4.467869
9,60,10,406.169882,4.467869,C,4.467869


## Combine Loadcases 

In [ ]:
PlyStressesStringer = PlyStressesStringerLC1.merge(PlyStressesStringerLC2, on=['Elements', 'PlyNumber'], suffixes=('_LC1', '_LC2'))
PlyStressesStringer = PlyStressesStringer.merge(PlyStressesStringerLC3, on=['Elements', 'PlyNumber'])
PlyStressesStringer

KeyError: 'PlyNumber'

## Output stringer

In [ ]:
PlyStressesStringer.to_excel(f'../data/{name}/output/stringerStrength.xlsx')